In [1]:
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [2]:
data = pd.read_csv('/Users/derekmcadam/GA/projects/project_5/client_project_5/cleandata/tweets.csv')